In [ ]:
import pandas as pd

# https://web.archive.org/web/20210902150937/https://eci.gov.in/files/file/13680-tamil-nadu-general-legislative-election-2021/

df_raw = pd.read_excel("../data/ac_wise_results_2021.xlsx", header=3)

df_raw = df_raw.rename(columns={
    "AC NO.": "ac_no",
    "AC NAME": "ac_name",
    "PARTY": "party",
    "TOTAL": "votes",
    "TOTAL ELECTORS": "total_electors"
})

df_raw = df_raw[
    ["ac_no", "ac_name", "party", "votes", "total_electors"]
]

In [12]:
df_clean = df_raw.copy()

df_clean["ac_no"] = pd.to_numeric(df_clean["ac_no"], errors="coerce")
df_clean["votes"] = pd.to_numeric(df_clean["votes"], errors="coerce")
df_clean["total_electors"] = pd.to_numeric(df_clean["total_electors"], errors="coerce")

df_clean = df_clean.dropna(subset=["ac_no", "votes", "total_electors"])

df_clean["ac_no"] = df_clean["ac_no"].astype(int)
df_clean["votes"] = df_clean["votes"].astype(int)
df_clean["total_electors"] = df_clean["total_electors"].astype(int)


In [13]:
df_turnout_2021 = (
    df_clean
    .groupby(["ac_no", "ac_name"], as_index=False)["votes"]
    .sum()
    .rename(columns={"votes": "total_votes_polled_2021"})
)


In [16]:
df_roll_2021 = (
    df_clean
    .groupby(["ac_no", "ac_name"], as_index=False)["total_electors"]
    .first()
    .rename(columns={"total_electors": "total_eligible_voters_2021"})
)


In [17]:
df_sorted = df_clean.sort_values(["ac_no", "votes"], ascending=[True, False])

df_winner = (
    df_sorted
    .groupby("ac_no")
    .nth(0)
    .reset_index()
    [["ac_no", "ac_name", "party", "votes"]]
    .rename(columns={
        "party": "winner_party",
        "votes": "votes_winner"
    })
)


In [19]:
df_runnerup = (
    df_sorted
    .groupby("ac_no")
    .nth(1)
    .reset_index()
    [["ac_no", "ac_name", "party", "votes"]]
    .rename(columns={
        "party": "runnerup_party",
        "votes": "votes_runnerup"
    })
)


In [20]:
df_results_2021 = df_winner.merge(
    df_runnerup,
    on=["ac_no", "ac_name"]
)

df_results_2021["victory_margin"] = (
    df_results_2021["votes_winner"]
    - df_results_2021["votes_runnerup"]
)


In [21]:
df_voters_2021 = (
    df_turnout_2021
    .merge(df_roll_2021, on=["ac_no", "ac_name"])
)

df_voters_2021["not_voted_2021"] = (
    df_voters_2021["total_eligible_voters_2021"]
    - df_voters_2021["total_votes_polled_2021"]
)


In [23]:
df_analysis = (
    df_results_2021
    .merge(df_voters_2021, on=["ac_no", "ac_name"])
)

df_analysis = df_analysis[
    [
        "ac_no",
        "ac_name",
        "winner_party",
        "runnerup_party",
        "votes_winner",
        "votes_runnerup",
        "victory_margin",
        "total_votes_polled_2021",
        "total_eligible_voters_2021",
        "not_voted_2021"
    ]
]

df_analysis


,ac_no,ac_name,winner_party,runnerup_party,votes_winner,votes_runnerup,victory_margin,total_votes_polled_2021,total_eligible_voters_2021,not_voted_2021
0,1,Gummidipoondi,DMK,PMK,126452,75514,50938,222069,281688,59619
1,2,Ponneri,INC,ADMK,94528,84839,9689,210354,267368,57014
2,3,Tiruttani,DMK,ADMK,120314,91061,29253,232624,291336,58712
3,4,Thiruvallur,DMK,ADMK,107709,85008,22701,214243,274982,60739
4,5,Poonamallee,DMK,PMK,149578,55468,94110,263736,358218,94482
...,...,...,...,...,...,...,...,...,...,...
229,230,Nagercoil,BJP,DMK,88804,77135,11669,184185,270845,86660
230,231,Colachal,INC,BJP,90681,65849,24832,182969,269287,86318
231,232,Padmanabhapuram,DMK,ADMK,87744,60859,26885,170156,240853,70697
232,233,Vilavancode,INC,BJP,87473,58804,28669,167836,250065,82229


In [24]:
import sqlite3
import os

os.makedirs("../db", exist_ok=True)
conn = sqlite3.connect("../db/tn_election.db")

print("Connected to SQLite")


Connected to SQLite


In [25]:
conn.execute("""
CREATE TABLE IF NOT EXISTS election_results_2021 (
    ac_no INTEGER PRIMARY KEY,
    ac_name TEXT,
    winner_party TEXT,
    runnerup_party TEXT,
    votes_winner INTEGER,
    votes_runnerup INTEGER,
    victory_margin INTEGER
);
""")


In [26]:
conn.execute("""
CREATE TABLE IF NOT EXISTS voters_2021 (
    ac_no INTEGER PRIMARY KEY,
    ac_name TEXT,
    total_votes_polled_2021 INTEGER,
    total_eligible_voters_2021 INTEGER,
    not_voted_2021 INTEGER
);
""")

conn.commit()
print("Tables created")


Tables created


In [27]:
df_results_2021.to_sql(
    "election_results_2021",
    conn,
    if_exists="replace",
    index=False
)


234

In [28]:
df_voters_2021.to_sql(
    "voters_2021",
    conn,
    if_exists="replace",
    index=False
)

print("Data inserted into SQL")


Data inserted into SQL


In [29]:
import pandas as pd

pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)


,name
0,election_results_2021
1,voters_2021


In [30]:
pd.read_sql(
    """
    SELECT
        e.ac_no,
        e.ac_name,
        e.winner_party,
        e.victory_margin,
        v.not_voted_2021
    FROM election_results_2021 e
    JOIN voters_2021 v
        ON e.ac_no = v.ac_no
    ORDER BY e.ac_no
    LIMIT 5;
    """,
    conn
)


,ac_no,ac_name,winner_party,victory_margin,not_voted_2021
0,1,Gummidipoondi,DMK,50938,59619
1,2,Ponneri,INC,9689,57014
2,3,Tiruttani,DMK,29253,58712
3,4,Thiruvallur,DMK,22701,60739
4,5,Poonamallee,DMK,94110,94482
